In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-27 15:46:08--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.25MB/s    in 0.2s    

2023-02-27 15:46:09 (5.25 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

1752932

##Transform the Data

##Create the "review_id_table"

In [6]:
# check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
df = df.withColumn("review_date",to_date(df.review_date))

In [8]:
# Check data types after changes
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:

review_id_df = df.select("review_id","customer_id","product_id","product_parent","review_date")
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

In [10]:
review_id_df.count()

1752932

##Create the "products" Table

In [12]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select("product_id","product_title").dropDuplicates(["product_id","product_title"])
products_df.show(truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                                                                                          |
+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|B00Y8QMQN6|Best Baby Teether - 4 Silicone Sensory Teething Ring Toys - Fun, Colorful and BPA-Free - Soothing Pain Relief and Drool Proof Infant Toys - Solve Teething Now!                                        |
|B002WN2BY6|The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit (Discontinued by Manufacturer)                                  

In [13]:
products_df.count()

160178

##Create the "customers" Table

In [14]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").count()
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.orderBy(customers_df["customer_count"].desc()).show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12084007|           192|
|   51591392|           141|
|   49633579|           140|
|   52457037|           114|
|   13103726|           112|
|   51983887|           109|
|   46137405|           105|
|   46106653|           105|
|   31461331|           105|
|   14389539|           102|
|   52097584|           101|
|   43715603|            99|
|   45725380|            98|
|   13404682|            98|
|   48428870|            93|
|   31546953|            90|
|   38555595|            89|
|   46912614|            87|
|   48384094|            86|
|   50785324|            85|
+-----------+--------------+
only showing top 20 rows



In [15]:
customers_df.count()

962117

##Create the "vine_table".

In [16]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id","star_rating","helpful_votes","total_votes","vine")
vine_df.orderBy(vine_df["total_votes"].desc()).show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1SYIVJAAO64D6|          5|         5245|       5471|   N|
| R4C1HKI1LDFYA|          5|         2777|       2803|   N|
|R3FLGAIB11MCAK|          3|         2482|       2739|   N|
|R2VC74WN54ZU7A|          4|         2621|       2738|   N|
|R3B5DVPWH12GAG|          5|         2123|       2183|   N|
|R1ELFB8E08WF4U|          4|         2016|       2166|   N|
|R3KMK5RWXY422Y|          5|         2008|       2115|   N|
|R2LBTZOH29KP8E|          2|         1923|       1978|   N|
|R2BD07MZG8KEBZ|          5|         1889|       1901|   N|
| RY4ZH927N5C4J|          5|         1824|       1873|   N|
|R1M6G8T9SFRYCQ|          4|         1714|       1864|   N|
|R34FW0UWRWQG9H|          3|         1811|       1820|   N|
| R5I5PIC436TAZ|          5|         1701|       1753|   N|
|R130WZMDSVEGRJ|          5|         168

In [17]:
vine_df.count()

1752932

##Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://:5432/"
config = {"user":"", "password": "", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='baby_review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='baby_products', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='baby_customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='baby_vine_table', mode=mode, properties=config)